# Introdução ao Pandas - Viagens do Governo | Análise de dados

*Esse notebook usa uma [versão tratada](https://github.com/anicelysantos/introducao-jornalismo-de-dados/blob/main/Tratamento%20de%20dados.ipynb) da base de [viagens de funcionários do governo](http://www.portaltransparencia.gov.br/viagens) disponibilizado no portal da transparência.*

In [1]:
import pandas as pd

Na importação do dataframe, vou mudar os dados das colunas com datas para o tipo datetime. Vou incluir no método do `read_csv` o `parse_dates` passando como parametro as colunas que quero alterar os dados.

In [19]:
df_viagens = pd.read_csv('viagens_tratado_2019.csv', encoding = 'latin-1', sep=';', parse_dates = ['Período - Data de início','Período - Data de fim'])

In [20]:
df_viagens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719261 entries, 0 to 719260
Data columns (total 19 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   Unnamed: 0                           719261 non-null  int64         
 1   Identificador do processo de viagem  719261 non-null  int64         
 2   Situação                             719261 non-null  object        
 3   Código do órgão superior             719261 non-null  int64         
 4   Nome do órgão superior               719261 non-null  object        
 5   Código órgão solicitante             719261 non-null  int64         
 6   Nome órgão solicitante               719261 non-null  object        
 7   CPF viajante                         719257 non-null  object        
 8   Nome                                 719261 non-null  object        
 9   Cargo                                502913 non-null  object        
 

**Contar a quantidade de um item em uma coluna**

In [4]:
df_viagens['país'].value_counts() #Passa a coluna por parametro

Brasil                       590392
Sem informação               115602
Estados Unidos da América      2225
Argentina                      1056
China                           881
                              ...  
Zimbábue                          1
Seichelles                        1
Timor Leste                       1
República do Sudão do Sul         1
Maldivas                          1
Name: país, Length: 170, dtype: int64

**Somar os valores de uma coluna**

In [5]:
valor_passagens = df_viagens['Valor passagens'].sum()
print(f'Valor de R$ {valor_passagens:.2f}')

Valor de R$ 429356157.93


**Somar valores por categoria**<br><br>
A sintaxe do `groupby()` pede como parametro **primeiro a coluna que possui os itens para agrupamento** e depois **a coluna que precisa trazer a quantidade** seguido do método que precisa pra trazer a quantidade (somar, média, dividir...). No caso do exemplo abaixo, eu estou querendo agrupar por países para saber quanto gastou em passagens por cada país.


In [6]:
#O sorte_values() indica a ordem que deve aparecer. Eu quero que apareça em descendente, por isso coloquei o parametro ascendig 
#como false. Coloquei o round(1) para arredondar os valores. O head(10) é pra aparecer os 10 primeiros.

df_viagens.groupby(['país'])['Valor passagens'].sum().sort_values(ascending=False).round(1).head(10)

país
Brasil                       306597339.5
Sem informação                70916743.6
Estados Unidos da América      9749399.4
China                          7208724.3
França                         3029309.7
Argentina                      1987897.8
Suíça                          1944638.0
Canadá                         1813266.3
Portugal                       1784548.6
Alemanha                       1644550.8
Name: Valor passagens, dtype: float64

**Maior valor em uma coluna**

In [7]:
df_viagens['Valor passagens'].max()

155531.36

**Procurar um valor específico**<br>

No exemplo eu estou querendo saber o destino da viagem com maior valor da coluna 'Valor passagens'.

In [8]:
df_viagens[df_viagens['Valor passagens'] == 155531.36]

,Unnamed: 0,Identificador do processo de viagem,Situação,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,Nome,Cargo,Período - Data de início,Período - Data de fim,Motivo,Valor diárias,Valor passagens,Valor outros gastos,cidade_destino,estado,país
596252,596252,16154465,Realizada,24000,"Ministério da Ciência, Tecnologia, Inovações e...",20402,Agência Espacial Brasileira,***.493.141-**,CARLOS EDUARDO QUINTANILHA VAZ DE OLIVEIRA,TECNOLOGISTA,09/06/2019,16/06/2019,Compor a delegação brasileira que participará ...,"10761,28",155531.36,0,Londres,Sem informação,Reino Unido


**Saber detalhes de uma busca específica**<br>

No exemplo abaixo eu quero saber detalhes sobre o motivo da viagem ter sido tão cara

In [9]:
#O loc pede como parametro o indice da linha. Caso seja de uma coluna especifica, basta incluir um colchete com o nome da coluna

df_viagens.loc[596252]['Motivo']

'Compor a delegação brasileira que participará da Reunião Plenária da ISO  TC20/SC14 - Space Systems and Operations, no período de 09 a 16 de junho de 2019, em Londres - Inglaterra, trânsito já incluído, com ônus.'

**Agrupar por cidade**

In [10]:
df_viagens.groupby(['cidade_destino'])['Valor passagens'].sum().sort_values(ascending=False).round(1).head(20)

cidade_destino
Brasília                             90418769.5
Informações protegidas por sigilo    48160610.6
Rio de Janeiro                       30880616.7
São Paulo                            26407457.1
Sem informação                       22756133.1
Manaus                                9059780.3
Recife                                7213480.2
Belo Horizonte                        6901541.9
Curitiba                              6668706.0
Belém                                 6296614.0
Porto Alegre                          6286268.8
Salvador                              5846874.7
Foz do Iguaçu                         5688308.2
São José dos Campos                   5025845.4
Florianópolis                         4945778.8
Fortaleza                             4707840.3
Wuhan                                 4555665.1
Natal                                 4340580.5
Campo Grande                          3740508.5
Cuiabá                                2934646.4
Name: Valor passagens, dt

No meio das cidades apareceu a cidade de Wuhan onde iniciou a pandemia do Covid-19. Preciso descobrir porque houveram tantas viagens para lá.

In [11]:
wuhan = df_viagens[df_viagens['cidade_destino'] == 'Wuhan']

Para entender o motivo das viagens, vou chamar o conteudo da coluna 'Motivo'

In [12]:
wuhan['Motivo'].value_counts()

Participação do proposto nos 7° Jogos Mundiais Militares (7° JMM) do Conselho Internacional do Esporte Militar (CISM), na cidade de Wuhan - China, no período de 18 a 27 de outubro de 2019.                                                                                                                                         434
Participar dos 7° Jogos Mundiais Militares (7° JMM) do Conselho Internacional do Esporte Militar (CISM), na cidade de Wuhan - China, no período de 18 a 27 de outubro de 2019.                                                                                                                                                        41
Participação de propostos nos 7° Jogos Mundiais Militares (7° JMM) do Conselho Internacional do Esporte Militar (CISM), na cidade de Wuhan - China, no período de 18 a 27 de outubro de 2019.                                                                                                                                         19
Participar, c

Percebi que o maior motivo de viagens foi devido aos jogos militares que aconteceu  nessa cidade.

**Agrupar a coluna 'Motivos'**

In [13]:
df_viagens['Motivo'].value_counts()

Informação protegida por sigilo nos termos da legislação vigente                                                                                                                                                                                                                                                                                                                      90186
ACOMPANHANTE DE PACIENTE INDÍGENA EM TRATAMENTO NA REFERENCIA III                                                                                                                                                                                                                                                                                                                      2731
PACIENTE INDÍGENA EM TRATAMENTO NA REFERENCIA III                                                                                                                                                                                               

**Busca por texto**<br><br>
Existem cerca de **90186** viagens que estão colocadas como sigilosas. Quero fazer uma busca por viagens que tem esse motivo. vou usar o metódo `str.contains()` e passar por parametro a palavra que eu estou procurando. Ele vai me retornar as linhas que tiverem essa palavra.

In [21]:
sigilo = df_viagens[df_viagens['cidade_destino'].str.contains('sigilo')]
sigilo.head()

,Unnamed: 0,Identificador do processo de viagem,Situação,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,Nome,Cargo,Período - Data de início,Período - Data de fim,Motivo,Valor diárias,Valor passagens,Valor outros gastos,cidade_destino,estado,país
603659,603659,15355834,Realizada,30000,Ministério da Justiça e Segurança Pública,30108,Departamento de Polícia Federal,***.000.000-**,Informações protegidas por sigilo,Informações protegidas por sigilo,2019-02-02,2019-02-02,Informação protegida por sigilo nos termos da ...,0,1492.03,0,Informações protegidas por sigilo,Sem informação,Sem informação
603660,603660,15355867,Realizada,30000,Ministério da Justiça e Segurança Pública,30108,Departamento de Polícia Federal,***.000.000-**,Informações protegidas por sigilo,Informações protegidas por sigilo,2019-02-02,2019-02-02,Informação protegida por sigilo nos termos da ...,0,1492.03,0,Informações protegidas por sigilo,Sem informação,Sem informação
603661,603661,15416039,Realizada,30000,Ministério da Justiça e Segurança Pública,30108,Departamento de Polícia Federal,***.000.000-**,Informações protegidas por sigilo,Informações protegidas por sigilo,2019-06-01,2019-07-01,Informação protegida por sigilo nos termos da ...,0,4304.31,0,Informações protegidas por sigilo,Sem informação,Sem informação
603662,603662,15416105,Realizada,30000,Ministério da Justiça e Segurança Pública,30108,Departamento de Polícia Federal,***.000.000-**,Informações protegidas por sigilo,Informações protegidas por sigilo,2019-06-01,2019-07-01,Informação protegida por sigilo nos termos da ...,0,4304.31,0,Informações protegidas por sigilo,Sem informação,Sem informação
603668,603668,15528952,Realizada,30000,Ministério da Justiça e Segurança Pública,30108,Departamento de Polícia Federal,***.000.000-**,Informações protegidas por sigilo,Informações protegidas por sigilo,2019-01-01,2019-05-01,Informação protegida por sigilo nos termos da ...,"734,04",1267.96,0,Informações protegidas por sigilo,Sem informação,Sem informação


Quero saber qual ministério mais coloca sigilo nas suas viagens

In [15]:
sigilo['Nome do órgão superior'].value_counts()

Ministério da Justiça e Segurança Pública              85384
Sem informação                                          4077
Controladoria-Geral da União                             651
Presidência da República                                  47
Ministério da Educação                                    11
Ministério da Agricultura, Pecuária e Abastecimento       11
Ministério das Relações Exteriores                         2
Ministério de Minas e Energia                              1
Advocacia-Geral da União                                   1
Ministério da Defesa                                       1
Name: Nome do órgão superior, dtype: int64

Agora que eu sei que foi o Ministério da Justiça quem fez o maior número de viagens secretas, quero saber quais órgão lá dentro que mais viajaram assim.

In [18]:
sigilo.groupby(['Nome do órgão superior'])['Nome órgão solicitante'].value_counts().sort_values(ascending=False)

Nome do órgão superior                               Nome órgão solicitante                                                           
Ministério da Justiça e Segurança Pública            Departamento de Polícia Federal                                                      49952
                                                     Departamento de Polícia Rodoviária Federal                                           35133
Sem informação                                       Sem informação                                                                        4077
Controladoria-Geral da União                         Controladoria-Geral da União - Unidades com vínculo direto                             651
Ministério da Justiça e Segurança Pública            Ministério da Justiça e Segurança Pública - Unidades com vínculo direto                299
Presidência da República                             Presidência da República - Unidades com vínculo direto                                  44
M

Quero investigar melhor a Policia Federal, para isso vou criar um dataframe só com informações relacionadas a ela.

In [23]:
sigilo.columns

Index(['Unnamed: 0', 'Identificador do processo de viagem', 'Situação',
       'Código do órgão superior', 'Nome do órgão superior',
       'Código órgão solicitante', 'Nome órgão solicitante', 'CPF viajante',
       'Nome', 'Cargo', 'Período - Data de início', 'Período - Data de fim',
       'Motivo', 'Valor diárias', 'Valor passagens', 'Valor outros gastos',
       'cidade_destino', 'estado', 'país'],
      dtype='object')

In [24]:
policia_federal = sigilo[sigilo['Código órgão solicitante'] == 30108] 

Vou renomear as colunas de data nesse novo dataframe

In [26]:
policia_federal.rename(columns = {'Período - Data de início':'data_inicio','Período - Data de fim':'data_fim'}, inplace = True)

C:\Users\anice\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Quero saber quantas viagens a Policia Federal fez por mês

In [33]:
policia_federal.data_inicio.dt.month.value_counts()

5     5144
3     4803
2     4673
11    4599
10    4245
1     4218
4     4074
8     3976
7     3915
6     3669
9     3478
12    3158
Name: data_inicio, dtype: int64

Descobri que foi no mês 5 (maio) que teve mais viagens feitas